In [1]:
import os
import pandas as pd

In [3]:
csv_file = "data/stock_training_2023-01-01_2024-12-31.csv"
if not os.path.exists(csv_file):
    raise FileNotFoundError(
        f"Please run data_fetcher.py to download the data first.")
else:
    df_all = pd.read_csv(csv_file)

In [ ]:
stocks = ["TSLA"]
for i, stock in enumerate(stocks):
        print(">>>>>>stock: ", stock)
        try:
            df = get_stock_df(df_all, stock)
            features, labels, dates = create_batch_feature(df)
            if np.isnan(features).any() or np.isnan(labels).any():
                print(f"NaN detected in {stock}")
                continue
            if np.isinf(features).any() or np.isinf(labels).any():
                print(f"INF detected in {stock}")
                continue
        except:
            print(f"Error in processing {stock}")
            continue
        if i == 0:
            all_features, all_labels, all_dates = features, labels, dates
        else:
            all_features = np.concatenate((all_features, features), axis=0)
            all_labels = np.concatenate((all_labels, labels), axis=0)
            all_dates = np.concatenate((all_dates, dates))
    print("total # of data samples: ", all_features.shape[0])

    train_loader, test_dataset, idx_test = split_train_test_data(
        all_features, all_labels, batch_size=128)

    model = PredictionModel(feature_len=all_features.shape[2],
                            seq_len=all_features.shape[1],
                            encoder_type=ENCODER_TYPE).to(device)
    model.load_state_dict(torch.load('./model/model.pth'))
    model.eval()
    criterion = CustomLoss()

    total_params = sum(p.numel() for p in model.parameters())
    print("total # of model params: ", total_params)
    predicted_labels, pr_table, dates_table = eval_model(
        model, criterion, test_dataset, all_dates[idx_test])
    print(pr_table)